# **Snakemake Demo**

Author: Jimmy Liu

Date: Feb 25th 2021

## **Introduction**
The purpose of this demo is to build upon the fundamentals of Snakemake and introduce additional concepts that may be useful for more advanced pipeline implementations. Thus far, we have seen how the snakemake workflow engine determines the flow of data from rule outputs and inputs, as well as the use of wildcards to circumvent the necessity of explicit file name definitions in every rule. Here, we will explore various methods to promote **flexability**, **generalization** and **modularization** of Snakemake pipelines.

## **Getting Started**
To install the necessary dependencies for this tutorial, first clone the GitHub repository followed by creating a new conda environment called `snakemake_demo2` by running the following commands in the terminal:

In [ ]:
git clone https://github.com/jimmyliu1326/PHL_snakemake_demo.git
cd PHL_snakemake_demo
conda env create -f conda_env.yml

Next, activate the conda environment with the following command line:

In [ ]:
conda activate snakemake_demo2

### 1. Generalization and Pipeline Configurations
* Data anlysis pipelines should be easily operable to readily process large-scale datasets
* Common standardized methods to define and parse data inputs
* Global config file to define pipeline parameters and variables

#### **Exercise 1.1**
Using a global `config.yaml` file to define pipeline parameters and variables that are accessible to Snakefile by referencing the `config` object.

Define `config.yaml`

In [ ]:
samples:
    samples_1
    samples_2
amr_db: card

Define `Snakefile`

In [ ]:
configfile: "config.yaml"

rule all:
    input:
        expand("abricate/{samples}_abricate_res.tsv", samples=config["samples"])

rule abricate:
    input:
        "data/{samples}.fasta"
    output:
        "abricate/{samples}_abricate_res.tsv"
    params:
        db=config["amr_db"]
    shell:
        "abricate --db {params.db} {input} > {output}"

#### **Exercise 1.2**
Define pipeline configurations via command lines without the creation of `config.yaml`

Define `Snakefile`

In [ ]:
rule all:
    input:
        expand("abricate/{samples}_abricate_res.tsv", samples=config["samples"])

rule abricate:
    input:
        "data/{samples}.fasta"
    output:
        "abricate/{samples}_abricate_res.tsv"
    params:
        db=config["amr_db"]
    shell:
        "abricate --db {params.db} {input} > {output}"

Run the following command line that includes pipeline configurations:

In [ ]:
snakemake --config samples=[sample_1,sample_2] amr_db=card --cores 4

#### **Exercise 1.3**
Instead of defining input data via `config.yaml`, an alternative method is to parse tabular input files, `samples.csv` and define the input dataset for analysis. A typical `samples.csv` has two columns: Sample Name and File Path; column headers are optional.

Example `samples.csv`:

sample_1,data/hello/sample_1.fasta
sample_2,data/world/sample_2.fasta

Define `Snakefile`
* Load the `samples.csv` as a pandas data frame
* Rename the columns if the csv file is headerless
* The list of sample names can be directly accessed from the pandas data frame
* Use of a lambda function that takes the `wildcards` object as input and returns the `input file path` from the pandas data frame

In [ ]:
import pandas as pd

samples_file=config["samples"]
samples_tbl=pd.read_csv(samples_file, header=None)
samples_tbl.columns=["Sample", "Path"]
samples_tbl=samples_tbl.set_index("Sample", drop=False)

rule all:
    input:
        expand("abricate/{samples}_abricate_res.tsv", samples=samples_tbl.Sample)

rule abricate:
    input:
        lambda wildcards: samples_tbl.Path[wildcards.samples]
    output:
        "abricate/{samples}_abricate_res.tsv"
    params:
        db=config["amr_db"]
    shell:
        "abricate --db {params.db} {input} > {output}"

Execute the snakemake pipeline using the following command line:

In [ ]:
snakemake --config samples=samples.csv amr_db=card --cores 4

#### **Exercise 1.4**
Instead of using lambda functions, an alternative method is to explicitly define a **helper function** that returns file path for a given sample in the pandas data frame

In [ ]:
import pandas as pd

samples_file=config["samples"]
samples_tbl=pd.read_csv(samples_file, header=None)
samples_tbl.columns=["Sample", "Path"]
samples_tbl=samples_tbl.set_index("Sample", drop=False)

def abricate_input(wildcards):
    sample=wildcards.samples
    path=samples_tbl.Path[sample]
    return path

rule all:
    input:
        expand("abricate/{samples}_abricate_res.tsv", samples=samples_tbl.Sample)

rule abricate:
    input:
        abricate_input
    output:
        "abricate/{samples}_abricate_res.tsv"
    params:
        db=config["amr_db"]
    shell:
        "abricate --db {params.db} {input} > {output}"

Execute the snakemake pipeline using the following command line:

In [ ]:
snakemake --config samples=samples.csv amr_db=card --cores 4

### 2. Flexibility and Conditionality
* May not always want to execute every components in the pipeline every time
* Empower end-users the flexibility to easily toggle pipeline flow and tailor the data analysis to their needs
* Different input data types may require different branches of analysis and processing
* Think of if..else statements

#### **Exercise 2.1**
**Scenario:** We are interested in constructing Salmonella genome assemblies from raw Illumina paired-end reads and typically a QC step is ran to filter out low quality reads prior to assembly, but occasionally someone has already done the QC for us, so there isn't a need to re-run read QC again in our pipeline. Is it possible to make read QC an optional step in our pipeline?

Define `snakefile`

In [ ]:
import pandas as pd
import os

samples_file=config["samples"]
samples_tbl=pd.read_csv(samples_file, header=None)
samples_tbl.columns=["Sample", "R1", "R2"]
samples_tbl=samples_tbl.set_index("Sample", drop=False)

def spades_input_R1(wildcards):
    sample=wildcards.samples
    if config["QC"] == 0:
        return samples_tbl.R1[sample]
    else:
        return os.path.join("read_QC", sample, sample+"_R1_trimmed.fastq.gz")

def spades_input_R2(wildcards):
    sample=wildcards.samples
    if config["QC"] == 0:
        return samples_tbl.R2[sample]
    else:
        return os.path.join("read_QC", sample, sample+"_R2_trimmed.fastq.gz")

rule all:
    input:
        expand("spades/{samples}", samples=samples_tbl.Sample)

rule read_QC:
    input:
        R1=lambda wildcards: samples_tbl.R1[wildcards.samples],
        R2=lambda wildcards: samples_tbl.R2[wildcards.samples]
    output:
        R1_trimmed=temp("read_QC/{samples}/{samples}_R1_trimmed.fastq.gz"),
        R2_trimmed=temp("read_QC/{samples}/{samples}_R2_trimmed.fastq.gz"),
        json=temp("read_QC/{samples}/fastp.json"),
        html=temp("read_QC/{samples}/fastp.html")
    threads: 4
    shell:
        """
        fastp -i {input.R1} -I {input.R2} -o {output.R1_trimmed} -O {output.R2_trimmed} \
            -w {threads} -j {output.json} -h {output.html}
        """

rule spades:
    input:
        R1=spades_input_R1,
        R2=spades_input_R2
    output:
        "spades/{samples}"
    threads: 4
    shell:
        "spades.py -1 {input.R1} -2 {input.R2} -o {output} -t {threads}"

Execute the following command line to **skip** the read QC step:

In [ ]:
snakemake --cores 4 --config samples=samples.csv QC=0

Execute the following command line to **run** the read QC step:

In [ ]:
snakemake --cores 4 --config samples=samples.csv QC=1

#### **Exercise 2.2**
**Scenario:** What if we have numerous samples of which only **some** raw reads (fastq) files require read QC. One simple solution is to create two different `samples.csv` files and use the snakemake pipeline written in `Exercise 2.1` using different parameters `QC=0/1` to analyze the samples separately. However, is it possible to implement conditional execution of certain components of the pipeline depending on the input type such that all samples can be processed in a single snakemake run?

Define `snakefile`

In [ ]:
import pandas as pd
import os

samples_file=config["samples"]
samples_tbl=pd.read_csv(samples_file, header=None)
samples_tbl.columns=["Sample", "R1", "R2", "Trim"]
samples_tbl=samples_tbl.set_index("Sample", drop=False)

def spades_input_R1(wildcards):
    sample=wildcards.samples
    if samples_tbl.Trim[sample] == "notrim":
        return samples_tbl.R1[sample]
    else:
        return os.path.join("read_QC", sample, sample+"_R1_trimmed.fastq.gz")

def spades_input_R2(wildcards):
    sample=wildcards.samples
    if samples_tbl.Trim[sample] == "notrim":
        return samples_tbl.R2[sample]
    else:
        return os.path.join("read_QC", sample, sample+"_R2_trimmed.fastq.gz")

rule all:
    input:
        expand("spades/{samples}", samples=samples_tbl.Sample)

rule read_QC:
    input:
        R1=lambda wildcards: samples_tbl.R1[wildcards.samples],
        R2=lambda wildcards: samples_tbl.R2[wildcards.samples]
    output:
        R1_trimmed=temp("read_QC/{samples}/{samples}_R1_trimmed.fastq.gz"),
        R2_trimmed=temp("read_QC/{samples}/{samples}_R2_trimmed.fastq.gz"),
        json=temp("read_QC/{samples}/fastp.json"),
        html=temp("read_QC/{samples}/fastp.html")
    threads: 4
    shell:
        """
        fastp -i {input.R1} -I {input.R2} -o {output.R1_trimmed} -O {output.R2_trimmed} \
            -w {threads} -j {output.json} -h {output.html}
        """

rule spades:
    input:
        R1=spades_input_R1,
        R2=spades_input_R2
    output:
        "spades/{samples}"
    threads: 4
    shell:
        "spades.py -1 {input.R1} -2 {input.R2} -o {output} -t {threads}"

Execute the following command line to **run** the Snakemake pipeline:

In [ ]:
snakemake --cores 4 --config samples=samples.csv

### 3. Modularization

Here we will explore the ability of Snakemake to integrate rules from multiple independent files (.smk) that can be reused for different pipeline implementations.

**Advantages of Modularization:**
* Improves readability of pipeline code
* Common analysis methods can be written as independent modules to facilitate easy integration

#### **Exercise 3.1**
We will use the `Snakefile` in `Ex2.1` as an example and attempt to decouple the rules into independent modules. The original `Snakefile` can be reduced down to simply contain the default target rule that specifies the final output of interest in our pipeline which is genome assemblies. All the other rules such as readQC and Spades assembly can be written as separate .smk files representing independent modules that can be readily loaded by other Snakefiles using `include`. The purpose of `common.smk` is to maintain a list of useful helper functions and parse data inputs.

Define `Snakefile`

In [ ]:
# load modules
include: "rules/common.smk"
include: "rules/readQC.smk"
include: "rules/assembly.smk"

rule all:
    input:
        expand("spades/{samples}", samples=samples_tbl.Sample)